In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class SingleHospital:
    def __init__(self, n_types, dist_lst):
        ''' Takes in number of pair types along with a list of functions that
        generate the number of people in that hospital with pair type.
        '''
        self.n_types = n_types
        self.dists = dist_lst
    def generate(self, batch_size):
        '''generate a report from this hospital'''
        X = np.zeros((batch_size, self.n_types))
        for i, dist in enumerate(self.dists):
            X[:, i] = dist(size=batch_size)
        return X
        
class ReportGenerator:
    def __init__(self, hos_lst, single_report_shape):
        self.hospitals = hos_lst
        self.single_shape = single_report_shape
    def generate_report(self, batch_size):
        X = np.zeros((batch_size,) + self.single_shape)
        for i, hos in enumerate(self.hospitals):
            X[:, i, :] = hos.generate(batch_size)
        yield X
        

In [3]:
def randint(low, high):
    return lambda size: np.random.randint(low, high, size)

def create_simple_generator(low_lst_lst, high_lst_lst, n_hos, n_types):
    hos_lst = []
    for h in range(n_hos):
        tmp_dist_lst = []
        for t in range(n_types):
            tmp_dist_lst.append(randint(low_lst_lst[h][t], high_lst_lst[h][t]))
        hos_lst.append(SingleHospital(n_types, tmp_dist_lst))
    gen = ReportGenerator(hos_lst, (n_hos, n_types))
    return gen

low_lst_lst = [[5, 10],[20, 40],[40, 80]]
high_lst_lst = [[10, 20], [40, 80], [80, 160]]
gen = create_simple_generator(low_lst_lst, high_lst_lst, 3, 2)
    


In [4]:
def create_u_mask(compat_lst, n_types, n_hos):
    '''
    Create mask matrix that will only reward valid matchings making the rest 0
    return shape (n_hos, n_types, n_types)
    '''
    mask = np.zeros([n_hos, n_types, n_types])
    print(mask.shape)
    for t1, t2 in compat_lst:
        mask[:, t1, t2] = 1.0
        mask[:, t2, t1] = 1.0
    return mask

In [5]:
learn_rate = 0.01
n_hos = 3
n_types = 2
batch_size = 10

n_features = n_hos * n_types
n_out = n_hos * n_types * n_types 

init = tf.keras.initializers.glorot_uniform()

num_layers = 5

neurons = [n_features, 100, 100, 100, n_out]

weights = []
biases = []

activation = [tf.nn.tanh] * (num_layers - 1) + [tf.nn.tanh]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# Creating input layer weights
print("Creating weights for layer: 1")
print([neurons[0], neurons[1]])
weights.append(tf.compat.v1.get_variable('w_a_1', [neurons[0], neurons[1]], initializer=init))


# Creating hidden layers
for i in range(1, num_layers - 2):
    print("Creating weights for layer: {}".format(i+1))
    print([neurons[i], neurons[i + 1]])
    weights.append(tf.compat.v1.get_variable('w_a_' + str(i+1), [neurons[i], neurons[i + 1]], initializer=init))

# need two outputs
print("Creating weights for layer: {}".format(num_layers - 1))
print([neurons[-2], neurons[-1]])
weights.append(tf.compat.v1.get_variable('w_a_' + str(num_layers - 1), [neurons[-2], neurons[-1]], initializer=init))
w_prime = tf.compat.v1.get_variable('wi_a_' + str(num_layers - 1), [neurons[-2], neurons[-1]], initializer=init)


# Biases
for i in range(num_layers - 2):
    print("Creating biases for layer: {}".format(i+1))
    biases.append(tf.compat.v1.get_variable('b_a_' + str(i+1), [neurons[i + 1]], initializer=init))

print("Creating biases for layer: {}".format(num_layers - 1))
biases.append(tf.compat.v1.get_variable('b_a_' + str(num_layers - 1), [neurons[-1]], initializer=init))
b_prime = tf.compat.v1.get_variable('bi_a_' + str(num_layers - 1), [neurons[-1]], initializer=init)




Creating weights for layer: 1
[6, 100]
Creating weights for layer: 2
[100, 100]
Creating weights for layer: 3
[100, 100]
Creating weights for layer: 4
[100, 12]
Creating biases for layer: 1
Creating biases for layer: 2
Creating biases for layer: 3
Creating biases for layer: 4


In [7]:
def feedforward(X):
    x_in = tf.reshape(X, [-1, neurons[0]])

    a = tf.nn.relu(tf.matmul(x_in, weights[0]) + biases[0], 'alloc_act_0')

    # push through hidden layers
    for i in range(1, num_layers - 2):
        a = tf.matmul(a, weights[i]) + biases[i]
        a = tf.nn.relu(a, 'alloc_act_' + str(i))

    # final layer
    pair = tf.matmul(a, weights[-1]) + biases[-1]
    pool = tf.matmul(a, w_prime) + b_prime

    # Softmax over pairs and total pool
    pair = tf.reshape(tf.nn.softmax(tf.reshape(pair, [-1, n_types * n_hos, n_types]), axis=-1), [-1, n_hos, n_types, n_types])
    pool = tf.reshape(tf.nn.softmax(tf.reshape(pool, [-1, n_types * n_hos, n_types]), axis=1), [-1, n_hos, n_types, n_types])

    ## Weighting softmax values ##

    # Weight softmax values by hospital's reported needs
    pair = tf.math.multiply(pair, tf.reshape(X, [-1, n_hos, n_types, 1]))

    # Sums over each hospitals for for all pair types then reshapes to allow broadcasting
    tot_reshaped = tf.reshape(tf.math.reduce_sum(tf.reshape(X, [-1, n_hos, n_types]), axis=1), [-1, 1, 1, n_types])

    # Weight softmax values by total available pairs in pool
    pool = tf.math.multiply(pool, tot_reshaped)

    # Floor of minimum of two allocations
    alloc = tf.math.floor(tf.math.minimum(pair, pool))
    return alloc
def create_misreports(x, curr_mis, self_mask):
    '''
    x and curr_mis should have dimensions (B, h, p)
    combined and og has dimensions (n_hos, batch_size, n_hos, n_types)
    '''
    tiled_curr_mis = tf.tile(tf.expand_dims(curr_mis, 0), [n_hos, 1, 1, 1])
    og_tiled = tf.reshape(tf.tile(x, [n_hos, 1, 1]), [n_hos, -1, n_hos, n_types])
    
    # Filter original reports to only keep reports from all other hospitals
    other_hos = og_tiled * (1 - self_mask)
    
    # Filter reports to only keep the mis reports
    only_mis = tiled_curr_mis * self_mask
    
    # Add the two to get inputs where only one hospital misreports
    combined = only_mis + other_hos
    return combined, og_tiled

def compute_util(alloc, mask, internal=None):
    if internal is None:
        return tf.reduce_sum(tf.multiply(alloc, mask), axis=(2, 3))
    else:
        mech_util = tf.reshape(tf.reduce_sum(tf.multiply(alloc, mask), axis=(2, 3)), [n_hos, -1, n_hos])
        return mech_util + compute_internal_util(internal)
def compute_internal_util(internal):
    '''
    internal has dim [n_hos, batch_size, n_hos, n_types]
    '''
    return tf.reduce_min(internal, axis=3)
def compute_internal(misreports, og):
    ''' Computes the difference between '''
    return (og - misreports) #maybe just keep the specific misreport bidder's difference

In [8]:
# This mask is to manipulate the reports and will only have 1 for the specific hospital misreporting
self_mask = np.zeros([n_hos, batch_size, n_hos, n_types])
self_mask[np.arange(n_hos), :, np.arange(n_hos), :] = 1.0

# This mask will only count the utility from the specific hospital that is misreporting
mis_u_mask = np.zeros((n_hos, batch_size, n_hos))
mis_u_mask[np.arange(n_hos), :, np.arange(n_hos)] = 1.0

# Mask to only count valid matchings 
u_mask = create_u_mask([(0,1)], n_types, n_hos)

(3, 2, 2)


In [9]:
X = tf.compat.v1.placeholder(tf.float32, [batch_size, n_hos * n_types], name='features')
curr_mis = tf.reshape(X, [batch_size, n_hos, n_types])

# Get misreports and also truthful original reports
misreports, og = create_misreports(tf.reshape(X, [batch_size, n_hos, n_types]), curr_mis, self_mask)

# Compute non-reported pairs by subtracting misreports from original
non_reported = compute_internal(misreports, og)



In [10]:
# Putting it together to get output
actual_alloc = feedforward(X)

# Run misreports through net to get allocation
misreport_alloc = feedforward(tf.reshape(misreports, [-1, n_hos * n_types]))

# Utility calculation
util = compute_util(actual_alloc, u_mask) # tf.reduce_sum(tf.multiply(alloc, mask), axis=(1, 2, 3))

# Compute utility from misreports for specific hospital misreporting only
mis_util = tf.reshape(compute_util(misreport_alloc, u_mask, non_reported), [n_hos, batch_size, n_hos]) * mis_u_mask



In [11]:
# optimizer = tf.train.AdamOptimizer(learn_rate)
init_op = tf.compat.v1.global_variables_initializer()

#test_in = np.random.randint(1, 100, size=(10, n_features))
sess = tf.compat.v1.InteractiveSession()

writer = tf.compat.v1.summary.FileWriter('./graphs', sess.graph)
sess.run(init_op)

In [25]:
test_in = np.reshape(next(gen.generate_report(batch_size)), (10, -1))
ex_util = sess.run(util, feed_dict={X:test_in})
ex_mis_util = sess.run(mis_util, feed_dict={X:test_in})
writer.close()

In [26]:
ex_util

array([[10., 31., 48.],
       [ 0.,  0., 43.],
       [ 1., 27., 20.],
       [ 3., 40., 18.],
       [ 0.,  2., 32.],
       [ 0., 10., 62.],
       [ 0., 13., 28.],
       [ 1., 19., 64.],
       [ 0.,  3., 43.],
       [ 0.,  5., 58.]], dtype=float32)

In [27]:
ex_mis_util

array([[[10.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 3.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0., 31.,  0.],
        [ 0.,  0.,  0.],
        [ 0., 27.,  0.],
        [ 0., 40.,  0.],
        [ 0.,  2.,  0.],
        [ 0., 10.,  0.],
        [ 0., 13.,  0.],
        [ 0., 19.,  0.],
        [ 0.,  3.,  0.],
        [ 0.,  5.,  0.]],

       [[ 0.,  0., 48.],
        [ 0.,  0., 43.],
        [ 0.,  0., 20.],
        [ 0.,  0., 18.],
        [ 0.,  0., 32.],
        [ 0.,  0., 62.],
        [ 0.,  0., 28.],
        [ 0.,  0., 64.],
        [ 0.,  0., 43.],
        [ 0.,  0., 58.]]], dtype=float32)

## Validating softmax over total pool ##

in_reshaped = np.reshape(test_in, [10, n_hos, n_types])
pool_tots = np.sum(in_reshaped, axis=1)

# Check for each pair type that the allocated does not exceed the total available
for sample in range(10):
    for p_type in range(n_types):
        print(round(np.sum(ex_pool[sample, :, :, p_type]) - pool_tots[sample, p_type]))

## Validating over hospital pair type need

# Check allocated amount to each hospital pair type does not exceed need.
for sample in range(10):
    for hos in range(n_hos):
        for p_type in range(n_types):
            print(round(np.sum(ex_pair[sample, hos, p_type, :]) - in_reshaped[sample, hos, p_type]))